In [1]:
#This is the conjob command that will auto post images/videos to social networks
#format
#cert_cron certificate_id channel <media_type>
#example: cert_cron.py AWS_SAA LinkedIn
#PDF: 20 questions (Github)

In [2]:
SHORT_VIDEO_CHANNELS = ['Youtube', 'Facebook', 'X', 'Tiktok']   #1 video = 3 images/questions
LONG_VIDEO_CHANNELS = ['Youtube']   #1 video = 20 images/questions
IMAGE_CHANNELS = ['LinkedIn', 'PInterest', 'Reddit']    #1 image = 3 questions
PDF_CHANNELS = ['Github']   #1 PDF = 4 pages * 5 questions

In [3]:
#1 question should have flag to indicate which channel(s) it appeared. for ex: "fb": 20250528
#we should post to social network once per day

In [4]:
import sys
import os
import importlib
import pymongo
from dotenv import load_dotenv
import subprocess
import time

# Get the current working directory of the notebook
notebook_dir = os.getcwd()
# Get the path to the parent directory
parent_dir = os.path.dirname(notebook_dir)

# Add the parent directory to sys.path if it's not already there
if parent_dir not in sys.path:
    sys.path.append(parent_dir)

In [5]:
load_dotenv(override=True) 

import thirdparty_sdk
importlib.reload(thirdparty_sdk)    #always get latest code
import exam_settings
importlib.reload(exam_settings) #always get latest code
import const
importlib.reload(const) #always get latest code
from thirdparty_sdk.linkedin import LinkedIn
importlib.reload(thirdparty_sdk.linkedin)    #always get latest code

<module 'thirdparty_sdk.linkedin' from '/Users/sang/Documents/Source/Python/python_webscrap/cert_exam/thirdparty_sdk/linkedin.py'>

In [6]:
db_client = pymongo.MongoClient(os.environ['DB_URI'])
db = db_client['db_certificates']

In [7]:
def create_img_with_3_images(documents):
    exam_settings.generate_image_portrait_from_file('img_3_question_template.html', './', 'test_3_q.png')

#test
#create_img_with_3_images(None)

In [8]:
#create video with only 1 image
#sample: ffmpeg -loop 1 -i input_image.jpg -i input_audio.mp3 -c:v libx264 -tune stillimage -c:a aac -b:a 192k -pix_fmt yuv420p -t 10 -shortest output_video.mp4
def create_video_from_image(img_path, output_filename):
    audio_path = os.path.abspath('audio/bg_audio_16sec.m4a')
    abs_output_path = os.path.abspath(output_filename)
    ffmpeg_command = [
        "/Users/sang/Downloads/SetupFiles/ffmpeg/ffmpeg",
        "-loop", "1",
        "-i", img_path,
        "-i", audio_path,
        "-c:v", "libx264",
        "-tune", "stillimage",
        "-c:a", "aac",
        "-b:a", "192k",
        "-pix_fmt", "yuv420p",
        "-t", "10",
        "-shortest",
        "-y", # Overwrite output file without asking
        abs_output_path
    ]

    try:
        result = subprocess.run(
            ffmpeg_command,
            # cwd=abs_image_folder, # This changes the directory for THIS command execution
            check=True,
            capture_output=True,
            text=True
        )
        print(f"Video '{output_filename}' created successfully at {abs_output_path}")
        return True
    except FileNotFoundError:
        print(f"Error: FFmpeg command not found. Make sure FFmpeg is installed and in your system's PATH.")
        return False
    except subprocess.CalledProcessError as e:
        print(f"Error: FFmpeg command failed with exit code {e.returncode}")
        return False
    except Exception as e:
        print(f"An unexpected error occurred: {e}")
        return False

In [1]:
def main(cert_symbol, channel):
    num_of_questions = 3    #default we export 3 questions
    if channel in LONG_VIDEO_CHANNELS or channel in PDF_CHANNELS:
        num_of_questions = 20
    #
    cert_metadata = exam_settings.get_cert_metadata(db, cert_symbol)
    #query random questions
    random_documents = exam_settings.query_random_questions(db, num_of_questions, channel, cert_metadata)
    if len(random_documents) == 0:
        print('Not enough question to export')
        return
    today_yyyymmdd = const.get_current_date_yyyymmdd()
    if channel in SHORT_VIDEO_CHANNELS or channel in IMAGE_CHANNELS:
        #create 1 image with multiple questions
        creation_result, img_filename = exam_settings.generate_1_img_multiple_questions(random_documents, cert_metadata, today_yyyymmdd)
        print(creation_result)
        print(img_filename)
        if creation_result == False:
            return
        if channel in SHORT_VIDEO_CHANNELS:
            #create video with 1 image only
            video_name = 'vid_multi_q_'+today_yyyymmdd + '.mp4'
            result_create_video = create_video_from_image(cert_metadata['img_m_q_folder_path']+img_filename, cert_metadata['img_m_q_folder_path'] + video_name)
            if result_create_video == False:
                return
            time.sleep(5)   #delay
            #todo: upload the video to social networks
        if channel in IMAGE_CHANNELS:   #1 image for 1 post
            answers_contents = []
            description_contents = []
            description_contents.append('#'+ cert_metadata['name'] + ' practice questions:\n')
            question_index = 1
            for doc in random_documents:
                str_index = str(question_index) + ') '
                print(str(str_index) + doc['question'])
                description_contents.append(str_index + doc['question'])
                answers_contents.append(str_index + doc['answer'])
                for explanation_key in doc['explanation']:
                    answers_contents.append(explanation_key + '. ' + doc['explanation'][explanation_key])
                question_index += 1

            # description_contents.append('\n' + 'Answers and explanations are in the comment')
            description_str = '\n'.join(description_contents)
            # print(description_str)
            answers_str = '\n'.join(answers_contents)
            description_str += '\n\n' + answers_str
            # print(answers_str)
            is_success_share = False
            if channel == 'LinkedIn':
                linkedin_obj = LinkedIn(os.environ['LI_URI'], os.environ['LI_REST_URI'], os.environ['LI_ACCESS_TOKEN'], os.environ['LI_VERSION'])
                #is_success_share = linkedin_obj.upload_and_share_img(cert_metadata, description_str, answers_str, cert_metadata['img_m_q_folder_path'] + img_filename)
            #update flag to shared questions
            if is_success_share == True:
                exam_settings.update_questions_posted(db, cert_metadata, channel, today_yyyymmdd, random_documents)

#test
if __name__ == '__main__':
    print(sys.argv)
    args = sys.argv
    if len(args) == 1:
        #todo there is no any param, get all certificates and post to all channels
        todo = 1

    # cert_symbol = args[1]   #certificate symbol
    # channel = args[2]
    ##AWS_SAA, AWS_SAP, AWS_CLF_C02, AWS_DVA_C02, AWS_MLA
    cert_symbol = 'AWS_SAP' #for testing
    channel = 'LinkedIn'
    #
    #main(cert_symbol, channel)  #exporting 1 image 3 questions

NameError: name 'sys' is not defined